In [44]:

import pandas as pd


eventos = pd.read_csv('logs_entrenamiento.csv')

notas = pd.read_csv('notas.csv')

column_names_df = notas.columns


column_names_eventos = eventos.columns

# Print the column names
print("Column names of 'df':", column_names_df)
print("Column names of 'eventos':", column_names_eventos)

# Rename a single column
notas.rename(columns={'Username': 'username'}, inplace=True)

print(notas)


Column names of 'df': Index(['Unnamed: 0', 'Username', 'Grade', 'Grade Scaled',
       'Quiz 1: Uso de modelos matemáticos en la economía',
       'Quiz 2: Motivación y Función de Producción',
       'Quiz 3: Preferencias y Toma de Decisiones',
       'Quiz 4: Salario y Efectos',
       'Quiz 5: Explicando Diferencias y Conclusión',
       'Quiz 6: Introducción y conceptos iniciales',
       'Quiz 7: Prediciendo el resultado de un juego',
       'Quiz 8: Evaluación de Resultados',
       'Quiz 9: Determinación de Asignaciones', 'Quiz 10: Desigualdad',
       'Quiz 11: Conceptos centrales', 'Quiz 12: Profundizando lo aprendido',
       'Quiz 13: Conceptos centrales', 'Quiz 14: Profundizando lo aprendido',
       'Quiz 15: Conceptos centrales', 'Quiz 16: Profundizando lo aprendido',
       'Quiz 17: Introducción',
       'Quiz 18: Oferta, demanda y equilibrio de mercado',
       'Quiz 19: Cambios en la oferta y la demanda',
       'Quiz 20: Excedentes y la mano invisible',
       'Quiz 2

In [45]:
#ACÁ SE TOMA INFO DE LOG_TRAIN PARA SUPLIR LA DATA DE NOTAS


# Convert the 'time_column' to a datetime object
eventos['time'] = pd.to_datetime(eventos['time'])

# Create new columns for month, day, and time
eventos['month'] = eventos['time'].dt.month
eventos['day'] = eventos['time'].dt.day
eventos['hora'] = eventos['time'].dt.strftime('%H:%M:%S')




#----------------------------------

# DURACION DE SESIONES (una sesion es un día con registro de ingreso)

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username', 'month', 'day'])['time'].agg(['min', 'max']).reset_index()

#
grouped['time_difference'] = (grouped['max'] - grouped['min']).dt.total_seconds() / 3600

# Merge the 'time_difference' column from the 'grouped' DataFrame into the 'eventos' DataFrame
eventos = eventos.merge(grouped[['username', 'month', 'day', 'time_difference']], on=['username', 'month', 'day'], how='left')



#----------------------------------

#ESTADISTICOS DE DURACION DE SESIONES

# Group the DataFrame by 'username' and calculate the average and standard deviation of 'time_difference'
grouped = eventos.groupby('username')['time_difference'].agg(['mean', 'std']).reset_index()

# Rename the columns for clarity
grouped.columns = ['username', 'average_time_difference', 'std_time_difference']

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'average_time_difference', 'std_time_difference']].drop_duplicates(), on='username', how='left')



#----------------------------------

#DURACION EN DÍAS DE SU USO DE EOL

# Extract the date part
eventos['date'] = eventos['time'].dt.date

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username'])['date'].agg(['min', 'max']).reset_index()

# Calculate the time difference in hours
grouped['duracionEOL'] = (grouped['max'] - grouped['min']).dt.days

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'duracionEOL']].drop_duplicates(), on='username', how='left')




In [46]:

#----------------------------------

#FRECUENCIA DE SESIONES POR MES


distinct_months = eventos['month'].unique()

# Create a new dataframe to store the results
result_data = {'username': eventos['username']}
for month in distinct_months:
    result_data[month] = eventos['username'].map(
        eventos[eventos['month'] == month].groupby('username')['day'].nunique()
    )

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(result_data)

# Rename multiple columns
result_df.rename(columns={'8': 'Num Sesiones de Agosto', '9': 'Num Sesiones de Septiembre', '10': 'Num Sesiones de Noviembre'}, inplace=True)


result_df.fillna(0, inplace=True)

# Merge the 'result_df' with the 'df_notas' dataframe based on the 'username' column
notas = notas.merge(result_df, on='username', how='left')

# Print the updated 'eventos' DataFrame
print(notas)


        Unnamed: 0                                           username  Grade  \
0                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
2                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
3                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
4                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
...            ...                                                ...    ...   
536764         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536765         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536766         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536767         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536768         613  714d13c3517b88defa5763d78fc0c840a4bc1ad6092c02...   0.00   

        Grade Scaled Quiz 1: Uso de mod

In [47]:
# Get unique values from a specific column
unique_values = eventos['grouped_event_type'].unique()
print(unique_values)

['/courses/(course-code)/course/'
 'edx.bi.course.upgrade.sidebarupsell.displayed'
 '/courses/(course-code)/progress' 'edx.ui.lms.link_clicked'
 '/courses/(course-code)/(uuid-course)/'
 '/courses/(course-code)/course_wiki'
 '/courses/(course-code)/wiki/(course-code)/'
 '/courses/(course-code)/discussion/(discussion-id)/'
 '/courses/(course-code)/jump_to/(block-code)@vertical@(uuid)'
 '/courses/(course-code)/courseware/(uuid1)/(uuid2)/(tab-n)'
 '/courses/(course-code)/xblock/block-v1:(block-code)@html+block@(uuid)/handler/publish_completion'
 'edx.ui.lms.sequence.next_selected'
 '/courses/(course-code)/courseware/(uuid-code1)/(uuid-code2)/(some-path)'
 'load_video'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/get_completion'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/goto_position'
 'seq_goto' 'play_video' 'pause_video' 'seek_video' 'page_close'
 '/courses/(course-code)/xb

In [48]:
#SE ENCUENTRAN LOS ESTADISTICOS DE LAS ACTIVIDADES QUE REALIZA EN CADA SESION
# IE CANTIDAD DE VECES QUE REALIZA ALGUNA ACTIVIDAD EN UNA SESION (MONTH, DAY)
# List of event types to create columns for
event_types = ['page_close', 'problem_graded', 'problem_check', 'problem_show', 'seg_prev',
               'seg_next', 'seg_goto', 'load_video', 'play_video', 'pause_video', 'speed_change_video']

# Iterate through the event types and create columns with boolean values
for event_type in event_types:
    eventos[event_type] = (eventos['grouped_event_type'] == event_type).astype(int)

grouped_eventos = eventos.groupby(['username', 'month', 'day'])[event_types].sum().reset_index()



# Group the sub_df DataFrame by 'username' and calculate the mean and standard deviation for each event type
agg_df = grouped_eventos.groupby('username').agg({
    'page_close': ['mean', 'std'],
    'problem_graded': ['mean', 'std'],
    'problem_check': ['mean', 'std'],
    'problem_show': ['mean', 'std'],
    'seg_prev': ['mean', 'std'],
    'seg_next': ['mean', 'std'],
    'seg_goto': ['mean', 'std'],
    'load_video': ['mean', 'std'],
    'play_video': ['mean', 'std'],
    'pause_video': ['mean', 'std'],
    'speed_change_video': ['mean', 'std']
}).reset_index()


# Assuming 'notas' is the DataFrame where you want to add the columns
notas = notas.merge(agg_df, on='username', how='left')

# Display the resulting 'notas' DataFrame


/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(
/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:3512: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  indexer = self._engine.get_indexer(target._get_engine_target())
/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:4574: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform e

In [49]:
#COUNT ACTIVIDADES PER CHAPTER (encontrar chapters que haya estudioado mas/menos)

grouped = eventos.groupby(['username', 'chapter'])['grouped_event_type'].count().reset_index()

pivoted_df = grouped.pivot(index='username', columns='chapter', values='grouped_event_type').fillna(0)

# Reset the index to make 'column1' a regular column
pivoted_df.reset_index(inplace=True)

# Rename columns for clarity
pivoted_df.columns.name = None

# Assuming 'notas' is the DataFrame where you want to add the columns
notas = notas.merge(pivoted_df, on='username', how='left')


In [50]:
#AGREGAR LABELS

label_df = eventos[['username', 'label']].drop_duplicates(subset='username')


merged_df = pd.merge(notas, label_df, on='username', how='inner')

merged_df.name = notas

/var/folders/x0/z6r3qq012s1ggpdmg3ps4ltr0000gn/T/ipykernel_3479/2118169417.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  merged_df.name = notas


In [51]:
#CREAMOS UN DICCIONARIO CON PARA PODER GUARDAR A LOS USUARIOS CON NOMBRES MAS PEQUEÑOS
import pandas as pd


# Genera nombres de usuario acortados automáticamente
notas['user_id'] = ['u' + str(i+1) for i in range(len(notas))]

# Crea un diccionario para mapear nombres acortados a nombres largos
diccionario_mapeo = dict(zip(notas['user_id'], notas['username']))

notas = notas.drop('username', axis=1)

column_order = ['user_id'] + [col for col in notas if col != 'user_id']
notas = notas[column_order]


#PARA ENCONTRAR A UN USUARIO SE USA LO SIGUIENTE
#nombre_acortado = 'u2'
#nombre_largo = diccionario_mapeo[nombre_acortado]

#print(f"Nombre largo correspondiente a '{nombre_acortado}': {nombre_largo}")


/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:2023: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.is_unique
/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [52]:
notas.sample(50)

,user_id,Unnamed: 0,Grade,Grade Scaled,Quiz 1: Uso de modelos matemáticos en la economía,Quiz 2: Motivación y Función de Producción,Quiz 3: Preferencias y Toma de Decisiones,Quiz 4: Salario y Efectos,Quiz 5: Explicando Diferencias y Conclusión,Quiz 6: Introducción y conceptos iniciales,...,"(speed_change_video, std)",0,1,2,3,4,5,6,7,none_page
434010,u434011,471,0.89,6.2,0,1.0,1.0,0.0,1.0,1.0,...,1.572795,20.0,118.0,245.0,526.0,285.0,434.0,436.0,244.0,412.0
130896,u130897,136,0.96,6.7,1.0,1.0,1.0,1.0,1.0,1.0,...,0.970143,1.0,295.0,318.0,728.0,539.0,930.0,394.0,369.0,235.0
465281,u465282,500,0.88,6.1,0,1.0,1.0,1.0,1.0,1.0,...,0.535303,0.0,27.0,242.0,191.0,205.0,95.0,295.0,146.0,273.0
277557,u277558,282,0.92,6.4,1.0,1.0,0.0,1.0,1.0,1.0,...,0.869718,20.0,76.0,96.0,86.0,99.0,103.0,87.0,118.0,253.0
89065,u89066,96,0.75,5.1,0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.0,103.0,6.0,163.0,53.0,9.0,50.0,101.0,235.0
72469,u72470,81,0.99,6.9,1.0,1.0,1.0,1.0,1.0,1.0,...,1.463224,0.0,552.0,184.0,710.0,218.0,189.0,214.0,166.0,384.0
418888,u418889,454,0.95,6.6,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.0,35.0,49.0,151.0,272.0,100.0,124.0,80.0,283.0
260161,u260162,268,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,12.0,177.0,57.0,47.0,79.0,34.0,118.0,79.0,570.0
186680,u186681,195,0.86,6.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.485071,24.0,48.0,67.0,126.0,163.0,153.0,42.0,145.0,322.0
385854,u385855,411,0.86,6.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.861770,22.0,99.0,168.0,204.0,326.0,153.0,311.0,126.0,272.0


In [53]:
notas.to_csv('data_para_entrenar.csv')

#PENDIENTE

- encontrar forma de incluir las variables de event type (conocer los distintos tipos de event type y sacar la frecuencia de cada uno en las sesiones y a partir de esto sacar estadisticos, ej en promedio cuantas veces pone pausa, a que vwlocidad ve ek video, repite mucho los mismos videos etc)
-quizas los estadísticos de la duracion de las sesiones podría ser especifico a cada mes
